# National Boundaries Dataset
> Εξετάζουμε την μορφή του National Boundaries Dataset και την μετατροπή της σε μορφη RDF.

Το National Boundary Database(2014)<cite data-cite="7747773/GSWXXBJF"></cite> είναι ενα dataset που περιγράφει τις διοικητικές μονάδες των Ηνωμένων Πολιτειών της Αμερικής . Για την μετατροπή των δεδομένων σε RDF data και την κατασκεύη του γράφου χρησιμοποιούμε το εργαλείο GeoTriples <cite>[2]</cite> .


Ας πάρουμε για παράδειγμα την πολιτεία της **Νέας Υόρκης** . Στο φάκελο data έχουμε το αρχείο gdb , που περιλαμβάνει η NBD για την πολιτεια .Αρχικα , θα κάνουμε χρήση των βιβλιοθηκών fiona <cite>[2]</cite> και geopandas <cite>[3]</cite> για την άναγνωση των δεδομένων του gdb.

In [ ]:
%matplotlib inline

In [ ]:
import fiona 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

## Νεα Υορκη
<img src="images/1280px-Flag_of_New_York.svg.png" alt="Flag" width="300"  />
<img src="images/New_York_in_United_States.svg" alt="State" width="300" />
Ξεκινάμε διαβάζοντας το αρχείο και εκτύπωνοντας τα δίαφορα layers.

In [ ]:
filepath = "./data/GOVTUNIT_New_York_State_GDB.gdb"

layers = fiona.listlayers(filepath)
print(layers)
print(len(layers))

Το αρχείο περιέχει 14 συνολικά layers . Απο αυτά τα 11 παρακάτω αντιστοιχούν σε γεωγραφικά δεδομένα , ενω τα υπόλοιπα περιέχουν πληροφορίες για τα μεταδεδομένα .

In [ ]:
gulayers = ['GU_CountyOrEquivalent', 'GU_IncorporatedPlace', 'GU_StateOrTerritory', 
            'GU_MinorCivilDivision', 'GU_Reserve', 'GU_NativeAmericanArea', 
            'GU_Jurisdictional', 'GU_UnincorporatedPlace', 'GU_PLSSFirstDivision', 
            'GU_PLSSSpecialSurvey', 'GU_PLSSTownship']

Εξετάζουμε την μορφη του κάθε layer.

Αρχικα , παρατηρούμε οτι το πιο πολυπληθές dataset είναι το GU_MinorCivilDivision με 1170 εγγραφές, ένω το λιγοτερο πολυπληθες - μεγαλυτερο του μηδεν - ειναι το StateOrTerritory με 7. 

In [ ]:
shapesdf = pd.DataFrame(columns=['Layer Name','Entries','Features'])
dfDict = {}
for idx,layer in enumerate(layers):
    gdf = gpd.read_file(filepath,layer=layer)
    dfDict[layer] = gdf
    shapesdf.loc[idx] = [layer,gdf.shape[0],gdf.shape[1]]
shapesdf = shapesdf.sort_values('Entries')
shapesdf.head(n=14)

## GU_StateOrTerritory


Το Dataset απoτελείται απο 7 εγγραφές . Κάθε μια απο τις εγγράφες αντιστοιχεί σε μια συνορεύουσα πολιτεία .Παρατηρούμε πολλές στήλες που αντιστοιχούν σε χρήσιμες πληροφορίες για κάθες πολιτεία . 
1. Την πηγή του dataset 
2. Την Ημερομηνία δημιουργίας 
3. Ο αριθμός FIPS που είναι μοναδικός για κάθε πολιτεία 
4. Το όνομα της πολιτείας 
5. Τον πληθησμό (δεν είναι συμπληρωμένο) 
6. Το μέγεθος 
7. την γεωμετρική αναπαράσταση.

In [ ]:
stateDf = dfDict['GU_StateOrTerritory']
stateDf.head(n=7)

Καλώντας την συνάρτηση plot() παράγουμε ενα γράφημα για με τις πληροφορίες του dataset .

In [ ]:
stateDf.plot(column='State_Name',figsize=(12, 20))

## GU_CountyOrEquivalent
Συνεχίζουμε μελετώντας την το layer GU_CountyOrEquivalent .Το dataset περιέχει παρόμοια features με το StateOrTerritory . Περιέχει , επίσης και εγγραφές απο γειτονικές πολιτείες ,(πχ. Wayne County ,PA)

In [ ]:
countyDf = dfDict['GU_CountyOrEquivalent']
countyDf.head(n=7)

In [ ]:
countyDf.loc[countyDf['State_Name'] == 'New York'].plot(column='County_Name',figsize=(12, 20))
plt.title('County or Equivalent Governmental Units in the State of New York')

## Σύνολο των δεδομένων 
Έχοντας κατεβάσει το αρχείο .zip μεγέθους 1.9GB απο τον [συνδεσμο](https://prd-tnm.s3.amazonaws.com/StagedProducts/GovtUnit/GDB/GovernmentUnits_National_GDB.zip) . Ανοίγουμε το αρχείο , οπως και με αυτό της Νεας Υόρκης και εξετάζουμε τα δεδομένα .


In [ ]:
bigpath = "./data/all/GovernmentUnits_National_GDB.gdb"

layers = fiona.listlayers(bigpath)
print(layers)
print(len(layers))

In [ ]:
gdf = gpd.read_file(bigpath,layer='GU_CountyOrEquivalent')
print(gdf.shape[0],gdf.shape[1])

In [ ]:
from tqdm.autonotebook import tqdm 
tqdm.pandas()

shapesdf = pd.DataFrame(columns=['Layer Name','Entries','Features'])
for i in tqdm(range(15)):
    gdf = gpd.read_file(bigpath,layer=layers[i])
    shapesdf.loc[idx] = [layers[i],gdf.shape[0],gdf.shape[1]]
shapesdf = shapesdf.sort_values('Entries')
shapesdf.head(n=14)

## Βιλιογραφία


  
    1. National Boundary Dataset (NBD) - ScienceBase-Catalog. https://www.sciencebase.gov/catalog/item/4f70b219e4b058caae3f8e19. Accessed 10 Apr 2020
  
  
    2. Gillies S (2011) Fiona
  
  
    3. Kelsey Jordahl, Joris Van den Bossche, Jacob Wasserman, et al (2019) geopandas/geopandas: v0.4.1. Zenodo
  
  
    4. Kyzirakos K, Vlachopoulos I, Savva D, et al (2014) GeoTriples: a tool for publishing geospatial data as RDF graphs using R2RML mappings. In: Proceedings of the 2014 International Conference on Posters & Demonstrations Track - Volume 1272. CEUR-WS.org, Riva del Garda, Italy, pp 393–396
  

